In [11]:
import pandas as pd
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.compose import ColumnTransformer
import keras

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import os

import math


from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime as dt

import warnings
warnings.filterwarnings('ignore')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # turn off deprecation warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# Prepare Data

In [2]:
df = pd.read_csv('../../../data/latest_minute_final_eth.csv')

In [3]:
df

,Unnamed: 0,datetime,open,high,low,close,volume,close_time,qav,num_trades,...,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg,DATE
0,1798,1641121080000,3743.44,3745.00,3743.18,3744.89,43.0514,1641121139999,1.611908e+05,223,...,-0.837393,-1.757981,119.486983,3750.150667,3746.859672,1.314016e-10,0.000285,0.864293,1.119881,2022-01-02
1,1799,1641121140000,3744.90,3745.20,3740.00,3743.16,126.4538,1641121199999,4.732167e+05,419,...,-0.834480,-1.877192,119.667585,3749.978667,3746.856653,6.570080e-11,0.000284,0.835955,1.110674,2022-01-02
2,1800,1641121200000,3743.17,3743.17,3741.02,3742.42,104.3907,1641121259999,3.906544e+05,317,...,-0.852333,-1.998954,119.792418,3749.788000,3746.849246,3.285040e-11,0.000284,0.867339,1.112477,2022-01-02
3,1801,1641121260000,3742.42,3744.70,3741.38,3744.70,132.6239,1641121319999,4.963977e+05,386,...,-0.759789,-2.107495,119.865138,3749.633000,3746.854211,1.642520e-11,0.000284,0.878419,1.083289,2022-01-02
4,1802,1641121320000,3744.69,3745.49,3744.38,3744.88,63.1470,1641121379999,2.364756e+05,318,...,-0.667460,-2.202847,119.877175,3749.482500,3746.859916,8.212600e-12,0.000283,0.925820,1.084700,2022-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685553,687467,1682265960000,1857.32,1857.32,1855.99,1856.65,668.0399,1682266019999,1.240130e+06,331,...,0.420757,-5.412923,122.622790,1858.787833,1868.069774,8.758326e+01,0.000274,0.999740,1.005976,2023-04-23
685554,687468,1682266020000,1856.64,1856.65,1855.73,1856.04,196.9905,1682266079999,3.656485e+05,268,...,0.353886,-5.362368,122.742167,1858.647833,1867.995796,5.647978e+01,0.000273,1.004418,0.995842,2023-04-23
685555,687469,1682266080000,1856.04,1857.24,1856.03,1856.89,180.5624,1682266139999,3.352345e+05,325,...,0.336879,-5.314243,122.812041,1858.522000,1867.926766,6.002647e+01,0.000272,1.012342,0.998162,2023-04-23
685556,687470,1682266140000,1856.90,1856.90,1856.57,1856.57,81.6304,1682266199999,1.515763e+05,151,...,0.311028,-5.269810,122.765255,1858.388167,1867.856192,7.134903e+01,0.000271,1.031932,1.007369,2023-04-23


In [4]:

df.drop(columns=['Unnamed: 0','datetime','DATE'], inplace=True)
df

,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,...,macd,macd_diff,macd_signal,mi,sma,wma,stc,trix,vi_pos,vi_neg
0,3743.44,3745.00,3743.18,3744.89,43.0514,1641121139999,1.611908e+05,223,30.3547,113652.512290,...,-2.595374,-0.837393,-1.757981,119.486983,3750.150667,3746.859672,1.314016e-10,0.000285,0.864293,1.119881
1,3744.90,3745.20,3740.00,3743.16,126.4538,1641121199999,4.732167e+05,419,25.6694,96058.138154,...,-2.711672,-0.834480,-1.877192,119.667585,3749.978667,3746.856653,6.570080e-11,0.000284,0.835955,1.110674
2,3743.17,3743.17,3741.02,3742.42,104.3907,1641121259999,3.906544e+05,317,71.7227,268413.373315,...,-2.851287,-0.852333,-1.998954,119.792418,3749.788000,3746.849246,3.285040e-11,0.000284,0.867339,1.112477
3,3742.42,3744.70,3741.38,3744.70,132.6239,1641121319999,4.963977e+05,386,78.9169,295368.126277,...,-2.867284,-0.759789,-2.107495,119.865138,3749.633000,3746.854211,1.642520e-11,0.000284,0.878419,1.083289
4,3744.69,3745.49,3744.38,3744.88,63.1470,1641121379999,2.364756e+05,318,24.7540,92701.073885,...,-2.870307,-0.667460,-2.202847,119.877175,3749.482500,3746.859916,8.212600e-12,0.000283,0.925820,1.084700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685553,1857.32,1857.32,1855.99,1856.65,668.0399,1682266019999,1.240130e+06,331,310.1621,575669.687980,...,-4.992166,0.420757,-5.412923,122.622790,1858.787833,1868.069774,8.758326e+01,0.000274,0.999740,1.005976
685554,1856.64,1856.65,1855.73,1856.04,196.9905,1682266079999,3.656485e+05,268,61.7580,114621.836832,...,-5.008483,0.353886,-5.362368,122.742167,1858.647833,1867.995796,5.647978e+01,0.000273,1.004418,0.995842
685555,1856.04,1857.24,1856.03,1856.89,180.5624,1682266139999,3.352345e+05,325,98.9928,183788.120926,...,-4.977364,0.336879,-5.314243,122.812041,1858.522000,1867.926766,6.002647e+01,0.000272,1.012342,0.998162
685556,1856.90,1856.90,1856.57,1856.57,81.6304,1682266199999,1.515763e+05,151,12.4738,23162.386580,...,-4.958782,0.311028,-5.269810,122.765255,1858.388167,1867.856192,7.134903e+01,0.000271,1.031932,1.007369


In [5]:
scaler = MinMaxScaler()


# Fit the scaler on your dataframe (let's say it's called df)
df_normalized = scaler.fit_transform(df)

# Convert the normalized data back to a dataframe
df_normalized = pd.DataFrame(df_normalized, columns=df.columns)

In [6]:

# Convert the DataFrame to a NumPy array
raw_data = df_normalized.values
close = df_normalized['close'].values
close = close.reshape((len(close),1))
# Display the NumPy array
print(type(raw_data))
print(raw_data.shape)
print(close.shape)

<class 'numpy.ndarray'>
(685558, 74)
(685558, 1)


# Prepare Model

In [7]:
print('Prep Data')
num_train_samples = int(0.6 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)


Prep Data
num_train_samples: 411334
num_val_samples: 171389
num_test_samples: 102835


In [8]:
# Set Parameters

# delay: time in future that will be predicted
delay = 6 * 60


# sampling rate: period between timesteps within the sequence
# Sequence with rate=1 : t1,t2...tn
# Sequence with rate=3 : t1,t3...tn*3
sampling_rate = 20

# sequence length: sequence lenght of each sample
half_day = 12 * 60
sequence_length = int(5 * half_day / sampling_rate)


# sequence_stride: period between sequences
# First sequence starts at t0
# Second sequence will start at t1 with sequence_stride=1 or at t5 with sequence_stride=5
sequence_stride = 5

#batch_size: Number of timeseries samples in each batch (except maybe the last one). 
#If None, the data will not be batched (the dataset will yield individual samples).
# Huge impact in performance.
# Tip, should be multiple of 8
batch_size = 32

# Understanding our parameters
msg = f"The timeseries will consist of batches containing {batch_size} sequences of {sequence_length} samples."

msg += f"\nFinally our target is {delay} timesteps in the future, and will have data from {sequence_length * sampling_rate} timesteps in the past"
print(msg)

The timeseries will consist of batches containing 32 sequences of 180 samples.
Finally our target is 360 timesteps in the future, and will have data from 3600 timesteps in the past


In [9]:
train_dataset = keras.preprocessing.timeseries_dataset_from_array(
                        raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=False, # Shouldn't the shuffle be set to 0?
                    seed=33,
                    batch_size=batch_size,
                    start_index=0,
                    end_index=num_train_samples)

print("Done Train")

val_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=False,
                    seed=33,
                    batch_size=batch_size,
                    start_index=num_train_samples,
                    end_index=num_train_samples + num_val_samples)

print("Done Validation")      

test_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=sampling_rate,
                    sequence_stride=sequence_stride,
                    sequence_length=sequence_length,
                    shuffle=False,
                    seed=33,
                    batch_size=batch_size,
                    start_index=num_train_samples + num_val_samples)

print("Done Test")

2023-04-24 08:46:24.643977: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 243509728 exceeds 10% of free system memory.
2023-04-24 08:46:24.997581: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 101462288 exceeds 10% of free system memory.


Done Train
Done Validation
Done Test


# Build and Train Model

In [10]:
inputs = keras.Input(shape=(sequence_length, raw_data.shape[-1]))
x = layers.LSTM(32)(inputs)


outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

callbacks = [
    keras.callbacks.ModelCheckpoint("models/lstm",
   save_best_only=True) 
]

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

history = model.fit(train_dataset,
                    epochs=30,
                    validation_data=val_dataset,
                    callbacks=[callbacks, early_stopping])


2023-04-24 08:46:25.515152: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-24 08:46:25.516805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-24 08:46:25.518279: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

NameError: name 'EarlyStopping' is not defined

In [ ]:
# Save the trained model to a file
model.save('lstm_model_1.h5')

# Understanding the Model

In [ ]:
import matplotlib.pyplot as plt
loss = history.history["mae"]
val_loss = history.history["val_mae"]
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, "bo", label="Training MAE")
plt.plot(epochs, val_loss, "b", label="Validation MAE")
plt.title("Training and validation MAE")
plt.legend()
plt.show()

# Looking at the data

In [ ]:
pred_dataset = keras.preprocessing.timeseries_dataset_from_array(
                    raw_data[:-delay],
                    targets=close[delay:],
                    sampling_rate=1,
                    sequence_stride=1,
                    sequence_length=180,
                    shuffle=False, # Shouldn't the shuffle be set to 0?
                    seed=33,
                    batch_size=32,
                    start_index=num_train_samples,
                    end_index=num_train_samples + num_val_samples
                    )




In [ ]:


predictions = model.predict(pred_dataset)

In [ ]:
len(close[num_train_samples+180:num_train_samples + num_val_samples+1])

In [ ]:
len(predictions)

In [ ]:
# Generate the predictions using your model and store it in a variable called 'predictions'

# Plot the actual data
plt.plot(range(len(predictions)), close[num_train_samples+180:num_train_samples + num_val_samples+1], label='Actual Data')

# Plot the predicted data
plt.plot(range(len(predictions)), predictions, label='Predicted Data')

# Add labels, title, and legend
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Actual vs Predicted Data')
plt.legend()

# Display the plot
plt.show()

In [ ]:
# Generate the predictions using your model and store it in a variable called 'predictions'


actual = close[num_train_samples+180:num_train_samples + num_val_samples+1]

# Plot the actual data
plt.plot(range(len(actual[0:24*5*60])), actual[0:24*5*60], label='Actual Data')

# Plot the predicted data
plt.plot(range(len(actual[0:24*5*60])), predictions[0:24*5*60], label='Predicted Data')

# Add labels, title, and legend
plt.xlabel('X-axis Label')
plt.ylabel('Y-axis Label')
plt.title('Actual vs Predicted Data')
plt.legend()

# Display the plot
plt.show()